# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [553]:
problem = '''You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem. '''

## 2. Ask for parameters

In [568]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [569]:
#Show the resopnse in a formatted way
Markdown(response.text)

### Decision Variables:

*  $x_1$ : Litres used of the first liquid component.
*  $x_2$ : Litres used of the second liquid component. 


# 2. Ask for objective

In [570]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [571]:
Markdown(response2.text)

## Objective Function:

maximize  $\ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2}$ 


# 3. Ask for constraints

In [572]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [573]:
Markdown(response3.text)

## Constraints:

1. **Tank Capacity:**  $x_1 + x_2 \leq 80$ 
2. **Stability Requirement:** $(x_1 - \frac{x_1 + x_2}{2})^2 \leq 10$ and $(x_2 - \frac{x_1 + x_2}{2})^2 \leq 10$
3. **Non-negativity:** $x_1 \geq 0$,  $x_2 \geq 0$ 


## 3. Generate the pyomo code

In [574]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [575]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, maximize, log, sqrt
from pyomo.opt import SolverFactory

# Create a concrete model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define the objective function
model.obj = Objective(expr=log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=maximize)

# Define constraints
model.tank_capacity = Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model using a suitable solver (e.g., IPOPT)
solver = SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)
print("Optimal solution (x1, x2):", model.x1(), ",", model.x2())
print("Optimal objective value:", model.obj())
```

**Explanation:**

* **Import necessary Pyomo components:** Import `ConcreteModel`, `Var`, `Objective`, `Constraint`, `maximize`, `log`, and `sqrt` for defining the model and its components.
* **Create a concrete model:** Initialize an instance of `ConcreteModel` to hold the optimization problem.
* **Define decision variables:** Declare `x1` and `x2` as non-negative real variables using `Var(domain=NonNegativeReals)`.
* **Define the objective function:** Define the objective using `Objective`. The expression represents the mathematical function to be maximized, using `log` for natural logarithm and exponentiation for the cubic root.
* **Define constraints:** Define the tank capacity and stability constraints as separate `Constraint` objects. Each constraint's expression represents the corresponding inequality.
* **Solve the model:** Create a solver instance (using IPOPT in this case) and solve the model.
* **Print the results:** Display the solver status, termination condition, optimal values of decision variables (`x1`, `x2`), and the optimal objective function value.

This code sets up and solves the mathematical optimization problem you provided using Pyomo, providing the optimal solution and objective value. You can modify the code to experiment with different solvers or adjust the problem parameters as needed. 


## 4. Input problem data and try running the generated code

In [576]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, maximize, log, sqrt
from pyomo.opt import SolverFactory

# Create a concrete model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define the objective function
model.obj = Objective(expr=log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=maximize)

# Define constraints
model.tank_capacity = Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model using a suitable solver (e.g., IPOPT)
solver = SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)
print("Optimal solution (x1, x2):", model.x1(), ",", model.x2())
print("Optimal objective value:", model.obj())

Status: ok
Termination condition: optimal
Optimal solution (x1, x2): 43.162277233647174 , 36.8377229631074
Optimal objective value: 2.4827783318454744


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [577]:
print(response.text)

### Decision Variables:

*  $x_1$ : Litres used of the first liquid component.
*  $x_2$ : Litres used of the second liquid component. 



In [578]:
print(response2.text)

## Objective Function:

maximize  $\ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2}$ 



In [579]:
print(response3.text)

## Constraints:

1. **Tank Capacity:**  $x_1 + x_2 \leq 80$ 
2. **Stability Requirement:** $(x_1 - \frac{x_1 + x_2}{2})^2 \leq 10$ and $(x_2 - \frac{x_1 + x_2}{2})^2 \leq 10$
3. **Non-negativity:** $x_1 \geq 0$,  $x_2 \geq 0$ 



In [580]:
print(response4.text)

```python
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, maximize, log, sqrt
from pyomo.opt import SolverFactory

# Create a concrete model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define the objective function
model.obj = Objective(expr=log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=maximize)

# Define constraints
model.tank_capacity = Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model using a suitable solver (e.g., IPOPT)
solver = SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)
print("Optimal solution (x1, x2):", model.x1(), ",", model.x2())
prin